In [2]:
import sys
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
%matplotlib inline

In [3]:
#parent_dir = os.path.abspath(r'D:')

#for GitLab
parent_dir = os.path.abspath(r'../data/')

concatenated_yaws_dir = os.path.join(parent_dir, 'processed_data', 'Concatenated_Yaws')
#inputoutput_dir = os.path.join(parent_dir, 'processed_data', 'InputOutput')
output_dir = os.path.join(parent_dir, 'processed_data', 'ModelingData')


## Adding New Features/Lables

In [ ]:
concatenated_yaws_dir = os.path.join(parent_dir, 'processed_data', 'Concatenated_Yaws')

all_files = [file for file in os.listdir(concatenated_yaws_dir) if file.endswith('.csv')]

if len(all_files) == 0:
    print("No CSV files found in the directory.")
else:
    for file in all_files:
        file_path = os.path.join(concatenated_yaws_dir, file)

        df = pd.read_csv(file_path)

        if 'Turbulence' in df.columns:
            print("Turbulence feature already added in", file)
        else:
            df['Turbulence'] = df['wind_speed_std'] / df['wind_speed_mean']
            df.to_csv(file_path, index=False)
            print("Turbulence feature added to", file)


## List of all Yaws

In [ ]:

concatenated_yaws_dir = os.path.join(parent_dir, 'processed_data', 'Concatenated_Yaws')

os.chdir(concatenated_yaws_dir) 
csv_files = glob.glob('*.csv')

concatenated_dataframes = []  

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    concatenated_dataframes.append(df)

concatenated_df = pd.concat(concatenated_dataframes)

print(concatenated_df)


## Preaping data for model training and testing

In [ ]:
big_train_df = pd.DataFrame()
big_test_df = pd.DataFrame()

for df in concatenated_dataframes:
    train_df = df.sample(frac=0.8, random_state=42)
    test_df = df.drop(train_df.index)

    big_train_df = pd.concat([big_train_df, train_df])
    big_test_df = pd.concat([big_test_df, test_df])

print("Big Training Dataset Shape:", big_train_df.shape)
print("Big Testing Dataset Shape:", big_test_df.shape)

## Exporting training and testing datasets

In [ ]:
os.makedirs(output_dir, exist_ok=True)

train_output_path = os.path.join(output_dir, 'big_train.parquet')
test_output_path = os.path.join(output_dir, 'big_test.parquet')

big_train_df.to_parquet(train_output_path)
big_test_df.to_parquet(test_output_path)